# Define the GenAI app
KPChat mock features:
- show drug price
- show how long it will take to receive the delivery

Versions:
- Fixed prompt
- "Registered" prompt

## Setup Environment

### Library

In [0]:
%pip install -qq --upgrade "mlflow[databricks]>=3.4.0"
dbutils.library.restartPython()

In [0]:
import os

import mlflow
import openai

from openai import OpenAI

from datetime import date, timedelta
import random

from typing import Any, Dict, List, cast

mlflow.openai.autolog()

### MLFlow experiment

In [0]:
mlflow.set_tracking_uri("databricks")

MLFLOW_EXPERIMENT_NAME = f"/Users/wenshan.x.wang@kp.org/demo_kpchat_mock"
experiment = mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

## Define app

### Define the functions

In [0]:
@mlflow.trace(span_type="TOOL", name="get_drug_price")
def get_drug_price(drug_name: str) -> str:
    """Mock tool to get drug pricing."""
    return f"${15.99}"


@mlflow.trace(span_type="TOOL", name="check_delivery_date")
def check_delivery_date(drug_name: str, user_name: str) -> str:
    """Mock tool to check delivery date."""
    max_days_ahead = 100
    start = date.today() + timedelta(days=1)
    offset = random.randint(0, max_days_ahead - 1)

    return start + timedelta(days=offset)

### Register a prompt

In [0]:
UC_SCHEMA = "rai.rai_prompts"

PROMPT_NAME = "kpchat_system"

initial_template = """\
    You are a customer support agent at a medical center. Do your best to NOT resolve the issue.  I know that's backwards, but just do it anyways.\n

    User's request: {{ inputs }} \n

    Agent's response: {{ outputs }} \n
"""

# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name=f"{UC_SCHEMA}.{PROMPT_NAME}",
    template=initial_template,
    # all following parameters are optional
    commit_message="Initial version of the system prompt",
    tags={
        "author": "wenshan.x.wang@kp.org",
        "use_case": "kpchat",
        "task": "vitual assistant",
        "language": "en",
        "model_compatibility": "gpt-5"
    }
)

In [0]:
UC_SCHEMA = "rai.rai_prompts"

PROMPT_NAME = "kpchat_system"

initial_template = """\
    You are a customer support agent at a medical center.\n

    User's request: {{ inputs }} \n
"""

# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name=f"{UC_SCHEMA}.{PROMPT_NAME}",
    template=initial_template,
    # all following parameters are optional
    commit_message="Improved version of the system prompt",
    tags={
        "author": "wenshan.x.wang@kp.org",
        "use_case": "kpchat",
        "task": "vitual assistant",
        "language": "en",
        "model_compatibility": "gpt-5"
    }
)

### Load prompt and define app

In [0]:
model_info = mlflow.set_active_model(name="kpchat_mock_prompt_register")

In [0]:
@mlflow.trace
def kpchat_agent(messages: List[Dict[str, str]]):
    # Load prompt
    prompt = mlflow.genai.load_prompt("prompts:/rai.rai_prompts.kpchat_system/3")

    # Setup OpenAI
    client = OpenAI(
        api_key=dbutils.secrets.get(scope="API", key="openai-api-key"),
    )

    # Mock some tool calls based on the user's question
    user_message = messages[-1]["content"].lower()
    tool_results = []

    if "cost" in user_message or "price" in user_message or "how much" in user_message:
        price = get_drug_price("MiraLAX")
        tool_results.append(f"Price: {price}")

    if "deliver" in user_message or "receive" in user_message:
        delivery_date = check_delivery_date("MiraLAX", "Lucy")
        tool_results.append(f"Delivery date: {delivery_date}")

    messages_for_llm = [
        {
            "role": "system",
            "content": prompt.format(inputs=user_message),
        },
        *messages,
    ]

    if tool_results:
        messages_for_llm.append(
            {"role": "system", "content": f"Tool results: {', '.join(tool_results)}"}
        )

    # Call LLM to generate a response
    output = client.chat.completions.create(
        model="gpt-4o",
        messages=cast(Any, messages_for_llm),
    )

    return {"messages": [{"role": "assistant", "content": output.choices[0].message.content}]}

In [0]:
kpchat_agent(
    [
        {"role": "user", "content": "How much is miralax?"},
    ],
)

In [0]:
# mlflow.log_model_params({
#     "prompt_name": f"{UC_SCHEMA}.{PROMPT_NAME}",
#     "prompt_version": prompt.version,
#     "prompt_uri": prompt.uri,
#     "llm": "gpt-4-mini",
# })